In [383]:
import pandas as pd
import os
os.chdir('F:\MSBAPM\Python\Assignment5(3)')

In [384]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## 1. Predicting Gender with Username
Some potential features of usernames: whether it has capital letters, whether it has digits, number of characters, number of vowels, first and last letters, etc. See http://www.nltk.org/book/ch06.html for some related code.

In [385]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [386]:
lb = LabelEncoder()

In [387]:
train.head()

,username,gender,status,description
0,Vimal20011,M,"{u'payment_verified': False, u'identity_verifi...",A team of 5 working on various projects relate...
1,sheom,M,"{u'payment_verified': True, u'identity_verifie...",We are an IT solution and service provider com...
2,ezbik,M,"{u'payment_verified': False, u'identity_verifi...",System administration is my work & hobby.
3,angelme,F,"{u'payment_verified': False, u'identity_verifi...",Good day! Thank you for taking some time to ch...
4,snitch1,M,"{u'payment_verified': False, u'identity_verifi...",I build good relation with clients and deliver...


### To create features using username

In [388]:
# Feature for 1st work being capital letter
gender_capital = []
for word in train['username']:
        if word[0].isupper() == True:
            gender_capital.append(1)
        else: gender_capital.append(0)

se = pd.Series(gender_capital)
train['Capital_Letter'] = se.values

#Feature to verify if username has digits
gender_digit = []
for word in train['username']:
       # if word[0].isdigit() == True:
        if any(i.isdigit() for i in word) == True:
            gender_digit.append(1)
        else: gender_digit.append(0)
gender_digit[:10]    
dig = pd.Series(gender_digit)
train['Hasdigit'] = dig.values

#Feature to calculate the legth of characters
gender_charlen = []
for word in train['username']:
      
        gender_charlen.append( len(word) - sum(c.isdigit() for c in word))

clen = pd.Series(gender_charlen)
train['charlen'] = clen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in train['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))

vlen = pd.Series(gender_vowlen)
train['vowlen'] = vlen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in train['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))
            
vlen = pd.Series(gender_vowlen)
train['vowlen'] = vlen.values

#code to identify last letter and 1st letter in the text
gender_firstletter = []
gender_lastletter = []
for word in train['username']:
      
        gender_firstletter.append(word[0])
        gender_lastletter.append(word[-1])
        
fl = pd.Series(gender_firstletter)
train['firstletter'] = fl.values

ll = pd.Series(gender_lastletter)
train['lastletter'] = ll.values

# To encode the gender column (target variable) to run models
train['gender'] = lb.fit_transform(train['gender'].values)
#feat.gender.value_counts()

#Encoding first letter and last letter of the column to avoid categorical values in the dataset
train['firstletter'] = lb.fit_transform(train['firstletter'].values)
train['lastletter'] = lb.fit_transform(train['lastletter'].values)

In [389]:
train.columns

Index(['username', 'gender', 'status', 'description', 'Capital_Letter',
       'Hasdigit', 'charlen', 'vowlen', 'firstletter', 'lastletter'],
      dtype='object')

In [390]:
feature_cols = ['Capital_Letter',  'Hasdigit', 'charlen', 'vowlen', 'firstletter', 'lastletter','gender']
feat = train[feature_cols]

In [391]:
feat.head()

,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,gender
0,1,1,5,2,21,1,1
1,0,0,5,2,44,47,1
2,0,0,5,2,30,45,1
3,0,0,7,3,26,39,0
4,0,1,6,1,44,1,1


### Naive bayes Classifier

In [392]:
X = feat.drop(['gender'], axis=1)
y = feat.gender
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X,y)
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size =0.40, random_state =123)
nb.fit(Xtrain,ytrain)
predictions = nb.predict(Xtest)

# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(ytest, predictions))

0.7864


### Logistic regression Model

In [393]:
X_log = feat.drop(['gender'], axis=1)
y_log = feat.gender
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X_log, y_log, random_state=0)

# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression
# instantiate model
logreg = LogisticRegression()
# fit model
model = logreg.fit(X1_train, y1_train)

# make class predictions for the testing set
y1_pred_class = model.predict(X1_test)

# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y1_test, y1_pred_class))

0.812539987204


### Decision Tree Model

In [394]:
#Import Library
from sklearn import tree
from sklearn.metrics import roc_auc_score
X_dec = feat.drop(['gender'], axis=1)
y_dec = feat.gender

# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X_dec, y_dec, test_size = 0.3,random_state=0)

# Create tree object 
decision = tree.DecisionTreeClassifier() 
# for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini  
# model = tree.DecisionTreeRegressor() for regression
decision

# Train the model using the training sets and check score
decision.fit(X1_train, y1_train)
decision.score(X1_train, y1_train)

#Predict Output
y_predict= decision.predict(X1_test)

from sklearn.metrics import accuracy_score
accuracy_score(y1_test, y_predict)

0.7002666666666667

## Test Data

In [395]:
test.columns

Index(['username', 'status', 'description'], dtype='object')

### creating features for the test set

In [396]:
# Feature for 1st work being capital letter
gender_capital = []
for word in test['username']:
        if word[0].isupper() == True:
            gender_capital.append(1)
        else: gender_capital.append(0)

se = pd.Series(gender_capital)
test['Capital_Letter'] = se.values

#Feature to verify if username has digits
gender_digit = []
for word in test['username']:
       # if word[0].isdigit() == True:
        if any(i.isdigit() for i in word) == True:
            gender_digit.append(1)
        else: gender_digit.append(0)
gender_digit[:10]    
dig = pd.Series(gender_digit)
test['Hasdigit'] = dig.values

#Feature to calculate the length of characters
gender_charlen = []
for word in test['username']:
      
        gender_charlen.append( len(word) - sum(c.isdigit() for c in word))

clen = pd.Series(gender_charlen)
test['charlen'] = clen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in test['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))

vlen = pd.Series(gender_vowlen)
test['vowlen'] = vlen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in test['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))
            
vlen = pd.Series(gender_vowlen)
test['vowlen'] = vlen.values

#code to identify last letter and 1st letter in the text
gender_firstletter = []
gender_lastletter = []
for word in test['username']:
      
        gender_firstletter.append(word[0])
        gender_lastletter.append(word[-1])
        
fl = pd.Series(gender_firstletter)
test['firstletter'] = fl.values

ll = pd.Series(gender_lastletter)
test['lastletter'] = ll.values

#Encoding
test['firstletter_enc'] = lb.fit_transform(test['firstletter'].values)
test['lastletter_enc'] = lb.fit_transform(test['lastletter'].values)

In [397]:
test.head()

,username,status,description,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,firstletter_enc,lastletter_enc
0,nazrulmadina,"{u'payment_verified': False, u'identity_verifi...",I am one of Self-employed person having more t...,0,0,12,5,n,a,39,33
1,SehrishWarraich,"{u'payment_verified': False, u'identity_verifi...",i am sehrish warraich.I do my job sincerelly a...,1,0,15,5,S,h,18,40
2,samadhinie,"{u'payment_verified': False, u'identity_verifi...","Since 2006, Web based solution provider (Web e...",0,0,10,5,s,e,44,37
3,ebottabi,"{u'payment_verified': False, u'identity_verifi...",Founder of a geolocation service developed on...,0,0,8,4,e,i,30,41
4,mrjimoy,"{u'payment_verified': False, u'identity_verifi...","Me? I am Jimmy, meaning honest, courageous, or...",0,0,7,2,m,y,38,57


In [398]:
feature_cols = ['Capital_Letter',  'Hasdigit', 'charlen', 'vowlen', 'firstletter_enc', 'lastletter_enc']
x_test = test[feature_cols]

### Running Naive Bayes model on test data

In [399]:
y_predict = nb.predict(x_test)
output_gender = []
for i in y_predict:
    if(i == 1): output_gender.append("M")
    else: output_gender.append("F")

output_gender[:10]

['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']

In [400]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test['username'], 'prediction':output_gender})
zz.to_csv('pred_uname_nb1.csv', index=False)

## Running Logistic model on test data

In [401]:
y_predict = model.predict(x_test)
output_gender = []
for i in y_predict:
    if(i == 1): output_gender.append("M")
    else: output_gender.append("F")

#np.unique(y_predict)

In [402]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test['username'], 'prediction':output_gender})
zz.to_csv('pred_uname_log.csv', index=False)

In [403]:
y_predict = decision.predict(x_test)
output_gender = []
for i in y_predict:
    if(i == 1): output_gender.append("M")
    else: output_gender.append("F")

np.unique(y_predict)

array([0, 1], dtype=int64)

In [404]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test['username'], 'prediction':output_gender})
zz.to_csv('pred_uname_dec.csv', index=False)

## 2. Predicting Gender with Description
The updated notebook for lecture 11 might be of some help, which now includes demo code for making predictions with NLTK classifier.

In [405]:
## read file from csv
train_desc = pd.read_csv('train.csv')
test_desc = pd.read_csv('test.csv')

In [406]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
import nltk
def preprocess(text):
    return [ps.stem(w) for w in word_tokenize(text.lower()) 
             if w not in string.punctuation and w not in stopwords.words('english')] 

In [407]:
#preprocess(train['description'][0])
train_df = train_desc[['gender','description']]
train_df.head()

,gender,description
0,M,A team of 5 working on various projects relate...
1,M,We are an IT solution and service provider com...
2,M,System administration is my work & hobby.
3,F,Good day! Thank you for taking some time to ch...
4,M,I build good relation with clients and deliver...


In [408]:
# To preprocess the description by tokenizing, removing stop words,punctuations and using equivalent stem words
description1 = list(train_df['description'])
gender1 = list(train_df['gender'])
tmp = []
for t, g in zip(description1, gender1):
     tmp.append((preprocess(t), g))

In [409]:
tmp[1]

(['solut',
  'servic',
  'provid',
  'compani',
  'expertis',
  'e-learn',
  'social',
  'media',
  'marketing.pleas',
  'visit',
  'compani',
  'url',
  'know'],
 'M')

### Extract Features

In [410]:
# Flattening the data
all_words = [w for words, c in tmp for w in words]
len(all_words)

298641

In [411]:
all_words[:10]

['team',
 '5',
 'work',
 'variou',
 'project',
 'relat',
 'data',
 'entri',
 'research',
 'work']

In [412]:
words_freq = nltk.FreqDist(all_words)

In [413]:
selected_words = [word for word, freq in words_freq.items() if freq>1]
print('Before:',len(words_freq), ', after:', len(selected_words))

Before: 36128 , after: 9953


In [414]:
token_data = [([w for w in words if w in selected_words], c) for words, c in tmp]

In [415]:
def extract_features(words, selected_words):
    ''' simply using words counts'''
    return nltk.FreqDist([w for w in words if w in selected_words])

In [416]:
feat_desc = [(extract_features(words,selected_words), c) for words, c in tmp]

In [417]:
feat_desc[:2]

[(FreqDist({'5': 1,
            'content': 1,
            'data': 1,
            'entri': 1,
            'project': 1,
            'relat': 1,
            'research': 1,
            'team': 1,
            'variou': 1,
            'work': 2,
            'write': 1}),
  'M'),
 (FreqDist({'compani': 2,
            'e-learn': 1,
            'expertis': 1,
            'know': 1,
            'media': 1,
            'provid': 1,
            'servic': 1,
            'social': 1,
            'solut': 1,
            'url': 1,
            'visit': 1}),
  'M')]

## Naive Bayes Classifier

In [418]:
# 2. Cross-Validation
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat_desc):
    train = [feat_desc[i] for i in train_idx]
    test = [feat_desc[i] for i in test_idx]
    classifier = nltk.NaiveBayesClassifier.train(train)   
    accu.append( nltk.classify.util.accuracy(classifier, test) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) ### Maximum Entropy classifier

accuracy: 0.5552
accuracy: 0.508
accuracy: 0.5992
accuracy: 0.6016
accuracy: 0.5532425940752602
CV mean accuracy: 0.563448518815


### Maximum Entropy classifier

In [419]:
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat_desc):
    train = [feat_desc[i] for i in train_idx]
    test = [feat_desc[i] for i in test_idx]
    classifier2_me = nltk.classify.MaxentClassifier.train(train, trace=3, max_iter=5)       
    accu.append( nltk.classify.util.accuracy(classifier2_me, test) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) 

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.814
             2          -0.34973        0.807
             3          -0.30039        0.800
             4          -0.28307        0.795
         Final          -0.27425        0.792
accuracy: 0.7848
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.813
             2          -0.46357        0.763
             3          -0.45449        0.701
             4          -0.45664        0.670
         Final          -0.45868        0.658
accuracy: 0.6296
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.811
             2          -0.41176        0.796
             3          -0.36109        0.777

### Scikitlearn Classifier - SVM

In [420]:
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat_desc):
    train = [feat_desc[i] for i in train_idx]
    test = [feat_desc[i] for i in test_idx]
    classifier_sk = SklearnClassifier(SVC(kernel='linear', C=10, random_state=1), sparse=True).train(train)       
    accu.append( nltk.classify.util.accuracy(classifier_sk, test) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu))

accuracy: 0.7664
accuracy: 0.7424
accuracy: 0.7544
accuracy: 0.712
accuracy: 0.7373899119295436
CV mean accuracy: 0.742517982386


## Test Data creation

In [421]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
import nltk
def preprocess(text):
    return [ps.stem(w) for w in word_tokenize(text.lower()) 
             if w not in string.punctuation and w not in stopwords.words('english')] 

In [422]:
#preprocess(train['description'][0])
#test_desc['gender_tst'] = test_desc.apply(lambda _: '', axis=1)
test_df = test_desc[['description']]
test_df.head()

,description
0,I am one of Self-employed person having more t...
1,i am sehrish warraich.I do my job sincerelly a...
2,"Since 2006, Web based solution provider (Web e..."
3,Founder of a geolocation service developed on...
4,"Me? I am Jimmy, meaning honest, courageous, or..."


In [423]:
# To preprocess the description by tokenizing, removing stop words,punctuations and using equivalent stem words
description_tst = list(test_df['description'])
tmp_test = []
for t in description_tst:
     tmp_test.append(preprocess(t))

In [424]:
# Flattening the data
all_words_tst = [w for words in tmp_test for w in words]
len(all_words_tst)

import nltk
words_freq_tst = nltk.FreqDist(all_words_tst)

selected_words_tst = [word for word, freq in words_freq_tst.items() if freq>1]
print('Before:',len(words_freq_tst), ', after:', len(selected_words_tst))

token_data = [([w for w in words if w in selected_words]) for words in tmp_test]

def extract_features(words, selected_words):
    ''' simply using words counts'''
    return nltk.FreqDist([w for w in words if w in selected_words_tst])

feat_tst = [(extract_features(words,selected_words_tst)) for words in tmp_test]

feat_tst[:2]

Before: 20142 , after: 5820


[FreqDist({'16': 1,
           'articl': 1,
           'blog': 1,
           'busi': 1,
           'comput': 2,
           'correspond': 1,
           'design': 1,
           'develop': 2,
           'etc': 1,
           'experi': 1,
           'one': 1,
           'page': 2,
           'person': 1,
           'program': 1,
           'self-employ': 1,
           'softwar': 1,
           'web': 2,
           'write': 1,
           'year': 1}),
 FreqDist({'better': 2,
           'data': 1,
           'entri': 1,
           'experienc': 1,
           'job': 2,
           'mission': 1,
           'next': 1,
           'one.i': 1,
           'previou': 1,
           'submit': 1,
           'time': 1})]

### Testing the data

In [425]:
#accuracy with sklearn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
import nltk

### Predicting data using Naive bayes classifier

In [426]:
#creating the list of predictions
feat_only = [(extract_features(words,selected_words_tst)) for words in tmp_test]
pred = [classifier.classify(row) for row in feat_only]

In [427]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test_desc['username'],'description':test_desc['description'], 'prediction':pred})
zz.to_csv('pred_desc_nb1.csv', index=False)

### Predicting using Maximum Enropy classifier

In [428]:
#creating the list of predictions
feat_only = [(extract_features(words,selected_words_tst)) for words in tmp_test]
pred_MEC = [classifier2_me.classify(row) for row in feat_only]

In [429]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test_desc['username'],'description':test_desc['description'], 'prediction':pred_SVM_SK})
zz.to_csv('pred_desc_entropy.csv', index=False)

### Predicting using SVM-Scikitlearn

In [430]:
#creating the list of predictions
feat_only = [(extract_features(words,selected_words_tst)) for words in tmp_test]
pred_SVM_SK = [classifier_sk.classify(row) for row in feat_only]

In [431]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test_desc['username'],'description':test_desc['description'], 'prediction':pred_SVM_SK})
zz.to_csv('pred_desc_svm.csv', index=False)

## 3. Predicting Gender with Username, Description, and Status
If you need to merge multiple dict-format features into one, check the following question: https://stackoverflow.com/questions/38987/how-to-merge-two-dictionaries-in-a-single-expression

In [432]:
# Parse Json format status as dictionary
from ast import literal_eval
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
status = train['status'].apply(literal_eval)

In [433]:
# Now you need to find a way to split the dictionary format status as multiple columns
status=status.apply(pd.Series)
status.head()

,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,True,True,False,False,False,False,True
1,True,True,True,False,True,False,True
2,False,True,False,False,False,False,True
3,True,True,False,False,False,True,True
4,False,True,False,False,False,False,True


In [434]:
train.columns

Index(['username', 'gender', 'status', 'description'], dtype='object')

### To create Features using username

In [435]:
# Feature for 1st work being capital letter
gender_capital = []
for word in train['username']:
        if word[0].isupper() == True:
            gender_capital.append(1)
        else: gender_capital.append(0)

se = pd.Series(gender_capital)
train['Capital_Letter'] = se.values

#Feature to verify if username has digits
gender_digit = []
for word in train['username']:
       # if word[0].isdigit() == True:
        if any(i.isdigit() for i in word) == True:
            gender_digit.append(1)
        else: gender_digit.append(0)
gender_digit[:10]    
dig = pd.Series(gender_digit)
train['Hasdigit'] = dig.values

#Feature to calculate the legth of characters
gender_charlen = []
for word in train['username']:
      
        gender_charlen.append( len(word) - sum(c.isdigit() for c in word))

clen = pd.Series(gender_charlen)
train['charlen'] = clen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in train['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))

vlen = pd.Series(gender_vowlen)
train['vowlen'] = vlen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in train['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))
            
vlen = pd.Series(gender_vowlen)
train['vowlen'] = vlen.values

#code to identify last letter and 1st letter in the text
gender_firstletter = []
gender_lastletter = []
for word in train['username']:
      
        gender_firstletter.append(word[0])
        gender_lastletter.append(word[-1])
        
fl = pd.Series(gender_firstletter)
train['firstletter'] = fl.values

ll = pd.Series(gender_lastletter)
train['lastletter'] = ll.values

# To encode the gender column (target variable) to run models
train['gender'] = lb.fit_transform(train['gender'].values)
#feat.gender.value_counts()

#Encoding first letter and last letter of the column to avoid categorical values in the dataset
train['firstletter'] = lb.fit_transform(train['firstletter'].values)
train['lastletter'] = lb.fit_transform(train['lastletter'].values)

In [436]:
train_merge = pd.concat([train,status],axis = 1)
train_merge.head()

,username,gender,status,description,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,Vimal20011,1,"{u'payment_verified': False, u'identity_verifi...",A team of 5 working on various projects relate...,1,1,5,2,21,1,True,True,False,False,False,False,True
1,sheom,1,"{u'payment_verified': True, u'identity_verifie...",We are an IT solution and service provider com...,0,0,5,2,44,47,True,True,True,False,True,False,True
2,ezbik,1,"{u'payment_verified': False, u'identity_verifi...",System administration is my work & hobby.,0,0,5,2,30,45,False,True,False,False,False,False,True
3,angelme,0,"{u'payment_verified': False, u'identity_verifi...",Good day! Thank you for taking some time to ch...,0,0,7,3,26,39,True,True,False,False,False,True,True
4,snitch1,1,"{u'payment_verified': False, u'identity_verifi...",I build good relation with clients and deliver...,0,1,6,1,44,1,False,True,False,False,False,False,True


### To encode the boolean values from statis column to binary

In [437]:
#Encoding
from sklearn.preprocessing import LabelEncoder

In [438]:
lb = LabelEncoder()

In [439]:
train_merge['firstletter'] = lb.fit_transform(train_merge['firstletter'].values)
train_merge['lastletter'] = lb.fit_transform(train_merge['lastletter'].values)
train_merge['deposit_made'] = lb.fit_transform(train_merge['deposit_made'].values)
train_merge['email_verified'] = lb.fit_transform(train_merge['email_verified'].values)
train_merge['facebook_connected'] = lb.fit_transform(train_merge['facebook_connected'].values)
train_merge['identity_verified'] = lb.fit_transform(train_merge['identity_verified'].values)
train_merge['payment_verified'] = lb.fit_transform(train_merge['payment_verified'].values)
train_merge['phone_verified'] = lb.fit_transform(train_merge['phone_verified'].values)
train_merge['profile_complete'] = lb.fit_transform(train_merge['profile_complete'].values)

In [440]:
train_merge.head(10)

,username,gender,status,description,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,Vimal20011,1,"{u'payment_verified': False, u'identity_verifi...",A team of 5 working on various projects relate...,1,1,5,2,21,1,1,1,0,0,0,0,0
1,sheom,1,"{u'payment_verified': True, u'identity_verifie...",We are an IT solution and service provider com...,0,0,5,2,44,47,1,1,1,0,1,0,0
2,ezbik,1,"{u'payment_verified': False, u'identity_verifi...",System administration is my work & hobby.,0,0,5,2,30,45,0,1,0,0,0,0,0
3,angelme,0,"{u'payment_verified': False, u'identity_verifi...",Good day! Thank you for taking some time to ch...,0,0,7,3,26,39,1,1,0,0,0,1,0
4,snitch1,1,"{u'payment_verified': False, u'identity_verifi...",I build good relation with clients and deliver...,0,1,6,1,44,1,0,1,0,0,0,0,0
5,ehabdigitalart,1,"{u'payment_verified': False, u'identity_verifi...","Over the last 12 years, I have developed a wid...",0,0,14,6,30,54,0,1,1,0,0,0,0
6,laarniandbuboy,0,"{u'payment_verified': False, u'identity_verifi...",WORK EXPERIENCESTraining SupervisorDirect Resp...,0,0,14,6,37,59,0,1,0,0,0,0,0
7,payzone,1,"{u'payment_verified': False, u'identity_verifi...",i'm graduate from engineering chemical faculty...,0,0,7,3,41,39,0,1,0,0,0,0,0
8,istratebogdan,1,"{u'payment_verified': False, u'identity_verifi...",Name: Istrate Bogdan PetrusNationality: Romani...,0,0,13,5,34,48,0,1,1,0,0,0,0
9,pam2489,0,"{u'payment_verified': False, u'identity_verifi...",My goal is to create long-lasting relationship...,0,1,3,1,41,9,1,1,0,0,0,0,0


In [441]:
##Convert dataframe into a dictionary
train_merge.columns

Index(['username', 'gender', 'status', 'description', 'Capital_Letter',
       'Hasdigit', 'charlen', 'vowlen', 'firstletter', 'lastletter',
       'deposit_made', 'email_verified', 'facebook_connected',
       'identity_verified', 'payment_verified', 'phone_verified',
       'profile_complete'],
      dtype='object')

In [442]:
gender2 = list(train_merge['gender'])
train_merge1 = train_merge.drop(['username','status','description','gender'],axis=1)

In [443]:
train_merge1.head()

,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,1,1,5,2,21,1,1,1,0,0,0,0,0
1,0,0,5,2,44,47,1,1,1,0,1,0,0
2,0,0,5,2,30,45,0,1,0,0,0,0,0
3,0,0,7,3,26,39,1,1,0,0,0,1,0
4,0,1,6,1,44,1,0,1,0,0,0,0,0


In [444]:
train_merge1.columns

Index(['Capital_Letter', 'Hasdigit', 'charlen', 'vowlen', 'firstletter',
       'lastletter', 'deposit_made', 'email_verified', 'facebook_connected',
       'identity_verified', 'payment_verified', 'phone_verified',
       'profile_complete'],
      dtype='object')

In [445]:
list1 = train_merge1.values.tolist()
list1[1]

[0, 0, 5, 2, 44, 47, 1, 1, 1, 0, 1, 0, 0]

### To process the description column to token, remove stop words, punctuations and to take stepwords

In [446]:
description1 = list(train_df['description'])
tmp1 = []
for t in description1:
    tmp1.append((preprocess(t)))

In [447]:
tmp1[1]

['solut',
 'servic',
 'provid',
 'compani',
 'expertis',
 'e-learn',
 'social',
 'media',
 'marketing.pleas',
 'visit',
 'compani',
 'url',
 'know']

### To append the features created from username, description and status

In [448]:
list_cons = []
for i in range(len(list1)):
    list_cons.append(list1[i]+tmp1[i])

list_cons[:1]

[[1,
  1,
  5,
  2,
  21,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  'team',
  '5',
  'work',
  'variou',
  'project',
  'relat',
  'data',
  'entri',
  'research',
  'work',
  'content',
  'write']]

### To append the gender column and create a dictionary

In [449]:
dict1 = []
for t, g in zip(list_cons, gender2):
     dict1.append((t, g))

In [450]:
dict1[:1]

[([1,
   1,
   5,
   2,
   21,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   'team',
   '5',
   'work',
   'variou',
   'project',
   'relat',
   'data',
   'entri',
   'research',
   'work',
   'content',
   'write'],
  1)]

In [451]:
#Flattening the lists
all_words_merge = [w for words, c in dict1 for w in words]
len(all_words)

298641

In [452]:
words_freq_merge = nltk.FreqDist(all_words_merge)
words_freq_merge['write']

1222

In [453]:
selected_word_merge = [word for word, freq in words_freq_merge.items() if freq>1]
print('Before:',len(words_freq), ', after:', len(selected_words))

Before: 36128 , after: 9953


In [454]:
token_data_merge = [([w for w in words if w in selected_word_merge], c) for words, c in dict1]

In [455]:
def extract_features(words, selected_words_merge):
    ''' simply using words counts'''
    return nltk.FreqDist([w for w in words if w in selected_word_merge])

In [456]:
feat_merge = [(extract_features(words,selected_word_merge), c) for words, c in dict1]

In [457]:
feat_merge[:2]

[(FreqDist({1: 5,
            5: 1,
            2: 1,
            21: 1,
            0: 5,
            'team': 1,
            '5': 1,
            'work': 2,
            'variou': 1,
            'project': 1,
            'relat': 1,
            'data': 1,
            'entri': 1,
            'research': 1,
            'content': 1,
            'write': 1}),
  1),
 (FreqDist({0: 5,
            5: 1,
            2: 1,
            44: 1,
            47: 1,
            1: 4,
            'solut': 1,
            'servic': 1,
            'provid': 1,
            'compani': 2,
            'expertis': 1,
            'e-learn': 1,
            'social': 1,
            'media': 1,
            'visit': 1,
            'url': 1,
            'know': 1}),
  1)]

### Naive Bayes Classifier

In [458]:
# 2. Cross-Valiation
from sklearn.model_selection import KFold
import numpy as np
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat_merge):
    train_nb = [feat_merge[i] for i in train_idx]
    test_nb = [feat_merge[i] for i in test_idx]
    classifier = nltk.NaiveBayesClassifier.train(train_nb)   
    accu.append( nltk.classify.util.accuracy(classifier, test_nb) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) ### Maximum Entropy classifier

accuracy: 0.5688
accuracy: 0.5888
accuracy: 0.5224
accuracy: 0.5712
accuracy: 0.603682946357
CV mean accuracy: 0.570976589271


### maximum Entropy Classifier

In [459]:
k_fold = KFold(n_splits=5, shuffle=True)
accu = []
for train_idx, test_idx in k_fold.split(feat_merge):
    train = [feat_merge[i] for i in train_idx]
    test = [feat_merge[i] for i in test_idx]
    classifier2_me = nltk.classify.MaxentClassifier.train(train, trace=3, max_iter=5)       
    accu.append( nltk.classify.util.accuracy(classifier2_me, test) )
    print('accuracy:', accu[len(accu)-1])    
print('CV mean accuracy:', np.mean(accu)) 

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.815
             2          -0.30708        0.815
             3          -0.26715        0.814
             4          -0.25560        0.809
         Final          -0.25000        0.804
accuracy: 0.7856
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.815
             2          -0.33043        0.815
             3          -0.27550        0.813
             4          -0.26102        0.806
         Final          -0.25477        0.802
accuracy: 0.7976
  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.813
             2          -0.29165        0.813
             3          -0.25035        0.812

### Creating Test Data

In [460]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
import nltk
def preprocess(text):
    return [ps.stem(w) for w in word_tokenize(text.lower()) 
             if w not in string.punctuation and w not in stopwords.words('english')] 

### Creating test features for status column

In [461]:
test = pd.read_csv('test.csv')
status_tst = test['status'].apply(literal_eval)
# Now you need to find a way to split the dictionary format status as multiple columns
status_tst=status_tst.apply(pd.Series)
status_tst.head()

,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,False,True,False,False,False,False,True
1,False,True,True,False,False,False,True
2,False,True,True,False,False,False,True
3,False,True,False,False,False,False,True
4,False,True,True,False,False,True,True


In [462]:
status_tst.shape

(2679, 7)

### Creating test features from username column

In [463]:
# Feature for 1st work being capital letter
gender_capital = []
for word in test['username']:
        if word[0].isupper() == True:
            gender_capital.append(1)
        else: gender_capital.append(0)

se = pd.Series(gender_capital)
test['Capital_Letter'] = se.values

#Feature to verify if username has digits
gender_digit = []
for word in test['username']:
       # if word[0].isdigit() == True:
        if any(i.isdigit() for i in word) == True:
            gender_digit.append(1)
        else: gender_digit.append(0)
gender_digit[:10]    
dig = pd.Series(gender_digit)
test['Hasdigit'] = dig.values

#Feature to calculate the length of characters
gender_charlen = []
for word in test['username']:
      
        gender_charlen.append( len(word) - sum(c.isdigit() for c in word))

clen = pd.Series(gender_charlen)
test['charlen'] = clen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in test['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))

vlen = pd.Series(gender_vowlen)
test['vowlen'] = vlen.values

#Feature to calculate the count of vowels
gender_vowlen = []
for word in test['username']:
      
        gender_vowlen.append(sum([1 for char in ''.join(word) if char in 'aeiouAEIOU']))
            
vlen = pd.Series(gender_vowlen)
test['vowlen'] = vlen.values

#code to identify last letter and 1st letter in the text
gender_firstletter = []
gender_lastletter = []
for word in test['username']:
      
        gender_firstletter.append(word[0])
        gender_lastletter.append(word[-1])
        
fl = pd.Series(gender_firstletter)
test['firstletter'] = fl.values

ll = pd.Series(gender_lastletter)
test['lastletter'] = ll.values

#Encoding
test['firstletter_enc'] = lb.fit_transform(test['firstletter'].values)
test['lastletter_enc'] = lb.fit_transform(test['lastletter'].values)

In [464]:
# Merge features for username and status
test_merge = pd.concat([test,status_tst],axis = 1)
test_merge.head()

,username,status,description,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,firstletter_enc,lastletter_enc,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,nazrulmadina,"{u'payment_verified': False, u'identity_verifi...",I am one of Self-employed person having more t...,0,0,12,5,n,a,39,33,False,True,False,False,False,False,True
1,SehrishWarraich,"{u'payment_verified': False, u'identity_verifi...",i am sehrish warraich.I do my job sincerelly a...,1,0,15,5,S,h,18,40,False,True,True,False,False,False,True
2,samadhinie,"{u'payment_verified': False, u'identity_verifi...","Since 2006, Web based solution provider (Web e...",0,0,10,5,s,e,44,37,False,True,True,False,False,False,True
3,ebottabi,"{u'payment_verified': False, u'identity_verifi...",Founder of a geolocation service developed on...,0,0,8,4,e,i,30,41,False,True,False,False,False,False,True
4,mrjimoy,"{u'payment_verified': False, u'identity_verifi...","Me? I am Jimmy, meaning honest, courageous, or...",0,0,7,2,m,y,38,57,False,True,True,False,False,True,True


In [465]:
#Encoding
from sklearn.preprocessing import LabelEncoder

In [466]:
lb = LabelEncoder()

In [467]:
test_merge['firstletter'] = lb.fit_transform(test_merge['firstletter'].values)
test_merge['lastletter'] = lb.fit_transform(test_merge['lastletter'].values)
test_merge['deposit_made'] = lb.fit_transform(test_merge['deposit_made'].values)
test_merge['email_verified'] = lb.fit_transform(test_merge['email_verified'].values)
test_merge['facebook_connected'] = lb.fit_transform(test_merge['facebook_connected'].values)
test_merge['identity_verified'] = lb.fit_transform(test_merge['identity_verified'].values)
test_merge['payment_verified'] = lb.fit_transform(test_merge['payment_verified'].values)
test_merge['phone_verified'] = lb.fit_transform(test_merge['phone_verified'].values)
test_merge['profile_complete'] = lb.fit_transform(test_merge['profile_complete'].values)

In [468]:
test_merge.head()

,username,status,description,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,firstletter_enc,lastletter_enc,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,nazrulmadina,"{u'payment_verified': False, u'identity_verifi...",I am one of Self-employed person having more t...,0,0,12,5,39,33,39,33,0,0,0,0,0,0,0
1,SehrishWarraich,"{u'payment_verified': False, u'identity_verifi...",i am sehrish warraich.I do my job sincerelly a...,1,0,15,5,18,40,18,40,0,0,1,0,0,0,0
2,samadhinie,"{u'payment_verified': False, u'identity_verifi...","Since 2006, Web based solution provider (Web e...",0,0,10,5,44,37,44,37,0,0,1,0,0,0,0
3,ebottabi,"{u'payment_verified': False, u'identity_verifi...",Founder of a geolocation service developed on...,0,0,8,4,30,41,30,41,0,0,0,0,0,0,0
4,mrjimoy,"{u'payment_verified': False, u'identity_verifi...","Me? I am Jimmy, meaning honest, courageous, or...",0,0,7,2,38,57,38,57,0,0,1,0,0,1,0


In [469]:
test_merge1 = test_merge.drop(['username','status','description'],axis=1)
test_merge1.head()

,Capital_Letter,Hasdigit,charlen,vowlen,firstletter,lastletter,firstletter_enc,lastletter_enc,deposit_made,email_verified,facebook_connected,identity_verified,payment_verified,phone_verified,profile_complete
0,0,0,12,5,39,33,39,33,0,0,0,0,0,0,0
1,1,0,15,5,18,40,18,40,0,0,1,0,0,0,0
2,0,0,10,5,44,37,44,37,0,0,1,0,0,0,0
3,0,0,8,4,30,41,30,41,0,0,0,0,0,0,0
4,0,0,7,2,38,57,38,57,0,0,1,0,0,1,0


In [470]:
list_tst = test_merge1.values.tolist()
len(list_tst)

2679

In [471]:
test_merge1.columns

Index(['Capital_Letter', 'Hasdigit', 'charlen', 'vowlen', 'firstletter',
       'lastletter', 'firstletter_enc', 'lastletter_enc', 'deposit_made',
       'email_verified', 'facebook_connected', 'identity_verified',
       'payment_verified', 'phone_verified', 'profile_complete'],
      dtype='object')

### To create feature for description in test

In [472]:
description_tst = list(test['description'])
tmp_tst = []
for t in description_tst:
    tmp_tst.append((preprocess(t)))

In [473]:
len(tmp_tst)

2679

### To merge features created for username, status and description in test

In [474]:
list_cons = []
for i in range(len(list_tst)):
    list_cons.append(list_tst[i]+tmp_tst[i])

In [475]:
# Flattening the data
all_words_tst = [w for words in list_cons for w in words]
len(all_words_tst)

import nltk
words_freq_tst = nltk.FreqDist(all_words_tst)

selected_words_tst = [word for word, freq in words_freq_tst.items() if freq>1]
print('Before:',len(words_freq_tst), ', after:', len(selected_words_tst))

token_data = [([w for w in words if w in selected_words]) for words in list_cons]

def extract_features(words, selected_words):
    ''' simply using words counts'''
    return nltk.FreqDist([w for w in words if w in selected_words_tst])

feat_tst = [(extract_features(words,selected_words_tst)) for words in list_cons]

feat_tst[:2]

Before: 20201 , after: 5879


[FreqDist({0: 9,
           12: 1,
           5: 1,
           39: 2,
           33: 2,
           'one': 1,
           'self-employ': 1,
           'person': 1,
           '16': 1,
           'year': 1,
           'experi': 1,
           'comput': 2,
           'program': 1,
           'softwar': 1,
           'develop': 2,
           'busi': 1,
           'correspond': 1,
           'articl': 1,
           'blog': 1,
           'write': 1,
           'web': 2,
           'page': 2,
           'design': 1,
           'etc': 1}),
 FreqDist({1: 2,
           0: 7,
           15: 1,
           5: 1,
           18: 2,
           40: 2,
           'job': 2,
           'submit': 1,
           'mission': 1,
           'better': 2,
           'next': 1,
           'time': 1,
           'previou': 1,
           'one.i': 1,
           'experienc': 1,
           'data': 1,
           'entri': 1})]

In [476]:
#accuracy with sklearn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
import nltk

### Predicting using Naive bayes

In [477]:
#creating the list of predictions
feat_only = [(extract_features(words,selected_words_tst)) for words in list_cons]
pred_merge_nb = [classifier.classify(row) for row in feat_only]

In [478]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test_desc['username'],'description':test_desc['description'], 'prediction':pred_merge_nb})
zz.to_csv('pred_merge_nb.csv', index=False)

### predciting using Maximum entropy

In [479]:
#creating the list of predictions
feat_only = [(extract_features(words,selected_words_tst)) for words in list_cons]
pred_merge_MEC = [classifier2_me.classify(row) for row in feat_only]

In [480]:
# support your predictions are stored in a list named pred_uname
zz = pd.DataFrame({'username':test_desc['username'],'description':test_desc['description'], 'prediction':pred_merge_MEC})
zz.to_csv('pred_merge_MEC.csv', index=False)

### In the 1st question:

### 1. Created features based on capital letters, firstname, lastname, length of vowels, legth of digits
### 2. Encoded the categorical variables to numerical using label encoder
### 3. Executed the models Naive bayes, logistic and decision tree gave better results

### Analysis: Logistic was overfitting, Naive bayes gave the best results

### In the 2nd question:

### 1. Created features for description by tokenization, removig stop words, puntuation and taking stem words
### 2. created model for SVM, Naive bayes and Entropy classifier.
### 3. Entropy classifier provided best results

### In the 3rd question

### 1. Created features for description by tokenization, removig stop words, puntuation and taking stem words
### 2.  Created features based on capital letters, firstname, lastname, length of vowels, legth of digits
### 3. split the status column for into multiple features and encoded the boolen values to binary.
### 4. Merged the features created in step 1 ,2 and 3
### 5. Created models Naive bayes and MEC
### 6. MEC gives the best results

### Extra Credit: Try Different Features and Models for Best Performance
Save your predictions as netid_1.csv, ..., netid_5.csv